## e621.net/e6ai.net JSON to `.txt` tags
----

This Python script is designed to process JSON files found within a specified directory and its subdirectories. Each JSON file is expected to contain data related to image posts, particularly sourced from online platforms such as image boards. The script parses these JSON files, extracts relevant information such as image URL, ratings, and tags, and generates caption files (`.txt`) based on this data.

Here's a breakdown of what the script does:

1. **Ignoring Tags**: The script defines a list of tags to be ignored during processing, such as "hi res", "shaded", etc.

2. **Processing Files**: The `process_file` function is responsible for processing each JSON file. It reads the JSON data, extracts the URL of the image file, determines its rating, and extracts tags associated with the image.

3. **Generating Caption File**: For each image, a caption file is generated with the same name as the image file but with a .txt extension. The rating of the image is written first, followed by processed tags.

4. **Processing Tags**: Tags are processed to replace underscores with spaces and to handle special cases such as artist tags. Ignored tags are filtered out.

In [ ]:
import os
import json
from rich.console import Console

console = Console()

# Define tags to be ignored
ignored_tags = [
    "hi res",
    "shaded",
    "tagme",
    "absurd res",
    "detailed",
    "dota",
    "creative commons",
    "cc-by-nc-nd"
]

def process_file(file_path):
    try:
        console.print(f"Processing file: [bold]{file_path}[/bold]")
        with open(file_path, "r") as f:
            data = json.load(f)

        # Parse the URL and generate filename
        post_data = data.get("post", {})
        file_data = post_data.get("file", {})
        url = file_data.get("url")
        if url:
            filename, ext = os.path.splitext(os.path.basename(url))

            # Create caption file
            caption_file = f"{filename}.txt"
            caption_path = os.path.join(os.path.dirname(file_path), caption_file)

            with open(caption_path, "w", encoding="utf-8") as f:
                console.print(f"Creating caption file: [bold]{caption_path}[/bold]")
                # Write rating
                rating = post_data.get("rating", "q")
                if rating == "s":
                    f.write("rating_safe, ")
                elif rating == "e":
                    f.write("rating_explicit, ")
                else:
                    f.write("rating_questionable, ")

                # Process tags
                tags = []
                tags_data = post_data.get("tags", {})
                for category, tags_list in tags_data.items():
                    for tag in tags_list:
                        # Replace underscores with spaces
                        tag = tag.replace("_", " ")
                        if tag.lower() not in ignored_tags:
                            processed_tag = process_tag(tag, category)
                            if processed_tag:
                                tags.append(processed_tag)

                # Check if there are any valid tags before writing
                if tags:
                    # Join tags with commas and write to file
                    tags_line = ", ".join(tags)
                    f.write(tags_line.strip())
                    console.print(f"Writing tags: [italic]{tags_line.strip()}[/italic]")

    except Exception as e:
        console.print(f"Error processing file: [bold]{file_path}[/bold]")
        console.print(e)

def process_tag(tag, category=None):
    if tag.isdigit() or all(part.isdigit() for part in tag.split(':')):
        return ""
    tag = tag.replace("_", " ")
    if category == "artist":
        return f"by {tag}"
    else:
        tag = tag.replace("(", "\\(").replace(")", "\\)")
        return tag

def recursive_process(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                process_file(file_path)

if __name__ == "__main__":
    root_directory = r"C:\Users\kade\Desktop\training_dir_staging"
    recursive_process(root_directory)